In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark spylon-kernel
!python3 -m spylon_kernel install


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.8/217.8 kB 13.4 MB/s eta 0:00:00
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
[InstallKernelSpec] Installed kernelspec spylon-kernel in /usr/local/share/jupyter/kernels/spylon-kernel


In [4]:
import org.apache.spark.sql.SparkSession
val spark= SparkSession.builder.appName("Spark with Scala Example").master("local[*]").getOrCreate()

import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@12cf9c99


**SparkScala For 5000 Sales Record**

In [5]:
val csvDF =spark.read.option("header","true").csv("/content/sample_data/5000 Sales Records.csv")
csvDF.show()
csvDF.cache()

+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|              Region|             Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|Central America a...|Antigua and Barbuda |      Baby Food|       Online|             M|12/20/2013|957081544| 1/11/2014|       552|    255.28|   159.42|    140914.56|  87999.84|    52914.72|
|Central America a...|              Panama|         Snacks|      Offline|             C|  7/5/2010|301644504| 7/26/2010|      2167|    152.58|    97.44|    330640.86| 211152.48|   119488.38|
|              Europe|      Czech Republic|  

csvDF: org.apache.spark.sql.DataFrame = [Region: string, Country: string ... 12 more fields]
res1: csvDF.type = [Region: string, Country: string ... 12 more fields]


In [6]:
csvDF.printSchema()

root
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Item Type: string (nullable = true)
 |-- Sales Channel: string (nullable = true)
 |-- Order Priority: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Units Sold: string (nullable = true)
 |-- Unit Price: string (nullable = true)
 |-- Unit Cost: string (nullable = true)
 |-- Total Revenue: string (nullable = true)
 |-- Total Cost: string (nullable = true)
 |-- Total Profit: string (nullable = true)



**Q. Display the products with atleast 2 occurences of 'a'.**

In [9]:
val reducedDF=csvDF.select("Item Type")
val productsWithTwoOrMoreA = reducedDF.filter(
  length(lower(col("Item Type"))) - length(regexp_replace(lower(col("Item Type")), "a", "")) >= 2
)
productsWithTwoOrMoreA.distinct().show()


+-------------+
|    Item Type|
+-------------+
|Personal Care|
+-------------+



reducedDF: org.apache.spark.sql.DataFrame = [Item Type: string]
productsWithTwoOrMoreA: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Item Type: string]


**Q.Display country in each region with highest units sold.**


In [12]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

// Start Spark session
val spark = SparkSession.builder()
  .appName("Spark DataFrame Analysis")
  .master("local[*]")
  .getOrCreate()

import spark.implicits._

val df = csvDF.select("Region", "Country", "Units Sold")
  .withColumn("Units Sold", $"Units Sold".cast("int"))

// CASE 1: WITHOUT PARTITIONING

val start1 = System.nanoTime()

val windowSpec1 = Window.partitionBy("Region").orderBy(desc("Units Sold"))

val topWithoutPartition = df
  .withColumn("row_num", row_number().over(windowSpec1))
  .filter($"row_num" === 1)
  .drop("row_num")

topWithoutPartition.show(false)

val end1 = System.nanoTime()
val duration1 = (end1 - start1) / 1e9d
println(s"\n⏱️ Time without partitioning: $duration1 seconds")
println("==================================================================")

// CASE 2: WITH PARTITIONING BY REGION

val start2 = System.nanoTime()

val repartitionedDF = df.repartition(col("Region"))

val windowSpec2 = Window.partitionBy("Region").orderBy(desc("Units Sold"))

val topWithPartition = repartitionedDF
  .withColumn("row_num", row_number().over(windowSpec2))
  .filter($"row_num" === 1)
  .drop("row_num")

topWithPartition.show(false)

val end2 = System.nanoTime()
val duration2 = (end2 - start2) / 1e9d
println(s"\n🚀 Time with partitioning: $duration2 seconds")


+---------------------------------+-----------------+----------+
|Region                           |Country          |Units Sold|
+---------------------------------+-----------------+----------+
|Asia                             |Thailand         |9991      |
|Australia and Oceania            |Samoa            |9990      |
|Central America and the Caribbean|Dominica         |9996      |
|Europe                           |United Kingdom   |9999      |
|Middle East and North Africa     |Libya            |9881      |
|North America                    |Greenland        |9898      |
|Sub-Saharan Africa               |Equatorial Guinea|9984      |
+---------------------------------+-----------------+----------+


⏱️ Time without partitioning: 1.116491543 seconds
+---------------------------------+-----------------+----------+
|Region                           |Country          |Units Sold|
+---------------------------------+-----------------+----------+
|Asia                             |Tha

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@12cf9c99
import spark.implicits._
df: org.apache.spark.sql.DataFrame = [Region: string, Country: string ... 1 more field]
start1: Long = 1363162198487
windowSpec1: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@9f9d074
topWithoutPartition: org.apache.spark.sql.DataFrame = [Region: string, Country: string ... 1 more field]
end1: Long = 1364278690030
duration1: Double = 1.116491543
start2: Long = 1364285626489
repartitionedDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Region: string, Country: string ... 1 more field]
windowSpec2:...


**Q.Display the unit price and unit cost of each item in ascending order.**

In [11]:
val priceCostDF=csvDF.select(col("Item Type"),col("Unit Price"),col("Unit Cost")).distinct().orderBy(col("Unit Price"),col("Unit Cost"))
priceCostDF.show()

+---------------+----------+---------+
|      Item Type|Unit Price|Unit Cost|
+---------------+----------+---------+
|        Clothes|    109.28|    35.84|
|         Snacks|    152.58|    97.44|
|     Vegetables|    154.06|    90.93|
|         Cereal|    205.70|   117.11|
|      Baby Food|    255.28|   159.42|
|           Meat|    421.89|   364.69|
|      Cosmetics|    437.20|   263.33|
|      Beverages|     47.45|    31.79|
|Office Supplies|    651.21|   524.96|
|      Household|    668.27|   502.54|
|  Personal Care|     81.73|    56.67|
|         Fruits|      9.33|     6.92|
+---------------+----------+---------+



priceCostDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Item Type: string, Unit Price: string ... 1 more field]


**SparkScala For XML-> csv Record**

In [13]:
import org.apache.spark.sql.functions._
import spark.implicits._


val retail_df = spark.read.option("header", "true").option("inferSchema", "true").csv("/content/sample_data/retailsales.csv")

val retail_df2 = retail_df.withColumn("DateParsed", to_timestamp(col("Sales_Date"), "MM/dd/yyyy"))

val retail_df3 = retail_df2.withColumn("Year", year(col("DateParsed"))).withColumn("Month", month(col("DateParsed"))).withColumn("Day", dayofmonth(col("DateParsed")))

retail_df3.select("Sales_Date","Amount", "Country", "Product", "Year", "Month", "Day").show(false)

+----------+------+-------+-------+----+-----+---+
|Sales_Date|Amount|Country|Product|Year|Month|Day|
+----------+------+-------+-------+----+-----+---+
|01/23/2005|350000|India  |bear   |2005|1    |23 |
|01/27/2005|380000|India  |visky  |2005|1    |27 |
|02/12/2005|450000|India  |Rum    |2005|2    |12 |
|01/23/2006|500000|USA    |bear   |2006|1    |23 |
|01/27/2006|550000|USA    |rum    |2006|1    |27 |
|02/12/2006|650000|USA    |Visky  |2006|2    |12 |
|01/23/2006|500000|China  |Beer   |2006|1    |23 |
|01/27/2006|550000|China  |Visky  |2006|1    |27 |
|02/12/2006|658000|China  |Rum    |2006|2    |12 |
+----------+------+-------+-------+----+-----+---+



import org.apache.spark.sql.functions._
import spark.implicits._
retail_df: org.apache.spark.sql.DataFrame = [Sales_Date: string, Amount: int ... 2 more fields]
retail_df2: org.apache.spark.sql.DataFrame = [Sales_Date: string, Amount: int ... 3 more fields]
retail_df3: org.apache.spark.sql.DataFrame = [Sales_Date: string, Amount: int ... 6 more fields]


**Q.yearly sales report**

In [16]:
val yr_sales_rep = retail_df3.groupBy("Year").agg(sum("Amount").alias("Yearly Sales")).orderBy("Year")
yr_sales_rep.show()

+----+------------+
|Year|Yearly Sales|
+----+------------+
|2005|     1180000|
|2006|     3408000|
+----+------------+



yr_sales_rep: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Year: int, Yearly Sales: bigint]


**Q.Yearly sum for all count**

In [17]:
val yearly_sum1 = retail_df3.groupBy("Year","Country").agg(sum("Amount").alias("Country Sales")).orderBy("Country")
yearly_sum1.show()

+----+-------+-------------+
|Year|Country|Country Sales|
+----+-------+-------------+
|2006|  China|      1708000|
|2005|  India|      1180000|
|2006|    USA|      1700000|
+----+-------+-------------+



yearly_sum1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Year: int, Country: string ... 1 more field]


**Q.Yearly sum for specified country**

In [18]:
val country_china = retail_df3.filter(col("Country") === "China").groupBy("Year").agg(sum("Amount").alias("China Sales")).orderBy("Year")
country_china.show()
country_china.coalesce(1).write.mode("overwrite").option("header","true").csv("/content/sample_data/country_china.csv")

+----+-----------+
|Year|China Sales|
+----+-----------+
|2006|    1708000|
+----+-----------+



country_china: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Year: int, China Sales: bigint]


**Q. yearly report dumping in the yr_sales_rep. Meanwhile we can apply all aggregation function here.**



In [19]:
import org.apache.spark.sql.functions._

val yr_sales_rep = retail_df3
  .groupBy("Year")
  .agg(
    sum("Amount").alias("Total_Sales"),
    avg("Amount").alias("Average_Sales"),
    min("Amount").alias("Min_Sales"),
    max("Amount").alias("Max_Sales"),
    count("Amount").alias("Transaction_Count"),
    stddev("Amount").alias("StdDev_Sales")
  )
  .orderBy("Year")

yr_sales_rep.show()


+----+-----------+-----------------+---------+---------+-----------------+------------------+
|Year|Total_Sales|    Average_Sales|Min_Sales|Max_Sales|Transaction_Count|      StdDev_Sales|
+----+-----------+-----------------+---------+---------+-----------------+------------------+
|2005|    1180000|393333.3333333333|   350000|   450000|                3|51316.014394468846|
|2006|    3408000|         568000.0|   500000|   658000|                6| 70313.58332498778|
+----+-----------+-----------------+---------+---------+-----------------+------------------+



import org.apache.spark.sql.functions._
yr_sales_rep: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Year: int, Total_Sales: bigint ... 5 more fields]


**Q.monthly sales report of a perticular year Dumping the data into mn_sales_rep**

In [20]:
val mn_sales_rep=retail_df3.filter(col("Year") === 2005).groupBy("Month").agg(sum("Amount").alias("montly_sales")).orderBy("Month")
mn_sales_rep.show()
mn_sales_rep.coalesce(1).write.mode("overwrite").option("header","true").csv("/content/sample_data/mn_sales_rep.csv")

+-----+------------+
|Month|montly_sales|
+-----+------------+
|    1|      730000|
|    2|      450000|
+-----+------------+



mn_sales_rep: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Month: int, montly_sales: bigint]


**Q.quarterly sales report of a particular year**

In [21]:
val quaterly_sales=retail_df3.withColumn("Quarter",quarter(to_date(concat_ws("-",col("Year"),col("Month"),lit(1)),"yyyy-M-d")))
quaterly_sales.show()
quaterly_sales.coalesce(1).write.mode("overwrite").option("header","true").csv("/content/sample_data/quaterly_sales.csv")


+----------+------+-------+-------+-------------------+----+-----+---+-------+
|Sales_Date|Amount|Country|Product|         DateParsed|Year|Month|Day|Quarter|
+----------+------+-------+-------+-------------------+----+-----+---+-------+
|01/23/2005|350000|  India|   bear|2005-01-23 00:00:00|2005|    1| 23|      1|
|01/27/2005|380000|  India|  visky|2005-01-27 00:00:00|2005|    1| 27|      1|
|02/12/2005|450000|  India|    Rum|2005-02-12 00:00:00|2005|    2| 12|      1|
|01/23/2006|500000|    USA|   bear|2006-01-23 00:00:00|2006|    1| 23|      1|
|01/27/2006|550000|    USA|    rum|2006-01-27 00:00:00|2006|    1| 27|      1|
|02/12/2006|650000|    USA|  Visky|2006-02-12 00:00:00|2006|    2| 12|      1|
|01/23/2006|500000|  China|   Beer|2006-01-23 00:00:00|2006|    1| 23|      1|
|01/27/2006|550000|  China|  Visky|2006-01-27 00:00:00|2006|    1| 27|      1|
|02/12/2006|658000|  China|    Rum|2006-02-12 00:00:00|2006|    2| 12|      1|
+----------+------+-------+-------+-----------------

quaterly_sales: org.apache.spark.sql.DataFrame = [Sales_Date: string, Amount: int ... 7 more fields]


In [22]:
val quaterly_df = retail_df3.withColumn("Quarter", quarter(col("DateParsed")))

val quaterly_year = quaterly_df.filter(col("Year") === 2005).groupBy("Quarter").agg(sum("Amount").alias("Quarterly Sales")).orderBy("Quarter")
quaterly_year.show()
quaterly_year.coalesce(1).write.mode("overwrite").option("header","true").csv("/content/sample_data/quarterly_year.csv")

+-------+---------------+
|Quarter|Quarterly Sales|
+-------+---------------+
|      1|        1180000|
+-------+---------------+



quaterly_df: org.apache.spark.sql.DataFrame = [Sales_Date: string, Amount: int ... 7 more fields]
quaterly_year: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Quarter: int, Quarterly Sales: bigint]


**Q.For each quartetr report for each year Write udf function for that.
And for all year**


In [23]:
import org.apache.spark.sql.functions.udf
val getQuarterUDF = udf((month: Int) => {
  if (month >= 1 && month <= 3) 1: java.lang.Integer
  else if (month >= 4 && month <= 6) 2: java.lang.Integer
  else if (month >= 7 && month <= 9) 3: java.lang.Integer
  else if (month >= 10 && month <= 12) 4: java.lang.Integer
  else null.asInstanceOf[java.lang.Integer]
})
val withQuarter = retail_df3.withColumn("Quarter", getQuarterUDF(col("Month")))
withQuarter.show()

+----------+------+-------+-------+-------------------+----+-----+---+-------+
|Sales_Date|Amount|Country|Product|         DateParsed|Year|Month|Day|Quarter|
+----------+------+-------+-------+-------------------+----+-----+---+-------+
|01/23/2005|350000|  India|   bear|2005-01-23 00:00:00|2005|    1| 23|      1|
|01/27/2005|380000|  India|  visky|2005-01-27 00:00:00|2005|    1| 27|      1|
|02/12/2005|450000|  India|    Rum|2005-02-12 00:00:00|2005|    2| 12|      1|
|01/23/2006|500000|    USA|   bear|2006-01-23 00:00:00|2006|    1| 23|      1|
|01/27/2006|550000|    USA|    rum|2006-01-27 00:00:00|2006|    1| 27|      1|
|02/12/2006|650000|    USA|  Visky|2006-02-12 00:00:00|2006|    2| 12|      1|
|01/23/2006|500000|  China|   Beer|2006-01-23 00:00:00|2006|    1| 23|      1|
|01/27/2006|550000|  China|  Visky|2006-01-27 00:00:00|2006|    1| 27|      1|
|02/12/2006|658000|  China|    Rum|2006-02-12 00:00:00|2006|    2| 12|      1|
+----------+------+-------+-------+-----------------

import org.apache.spark.sql.functions.udf
getQuarterUDF: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$5481/0x0000000841c35040@4509bb9c,IntegerType,List(Some(class[value[0]: int])),Some(class[value[0]: int]),None,true,true)
withQuarter: org.apache.spark.sql.DataFrame = [Sales_Date: string, Amount: int ... 7 more fields]
